[Reference](https://medium.com/@bragadeeshs/docker-networking-conducting-a-harmonious-campsite-with-streamlit-and-postgresql-62a39f698b36)

```
-- init.sql
CREATE TABLE camptable(
    campcolumn VARCHAR (100)
);
```

```
version: '3.8'
services:
  db:
    image: "postgres:13"
    environment:
      POSTGRES_DB: campdb
      POSTGRES_USER: camper
      POSTGRES_PASSWORD: camper123#
    volumes:
      - ./docker-entrypoint-initdb.d:/docker-entrypoint-initdb.d
    networks:
      - app_network

  app:  # Streamlit application service definition
    build: .  # Specify the directory of the Dockerfile
    environment:
      - DB_HOST=db  # Use the service name 'db' for database host
    ports:
      - "8501:8501"  # Default port for Streamlit
    depends_on:
      - db  # This service depends on 'db' service
    networks:
      - app_network

networks:
  app_network:
    driver: bridge
```

In [1]:
# streamlit_app.py
import streamlit as st
import psycopg2

st.title('Data Updater')
user_input = st.text_input("Enter data:")

if st.button('Update Data'):
    conn = psycopg2.connect(
        host="db",  # Note: Use the service name defined in docker-compose.yml
        database="campdb",
        user="camper",
        password="camper123#"
    )
    cursor = conn.cursor()
    cursor.execute("INSERT INTO camptable (campcolumn) VALUES (%s)", (user_input,))
    conn.commit()
    cursor.close()
    conn.close()
    st.success("Data updated successfully!")

```
# Dockerfile
FROM python:3.8
WORKDIR /app
COPY . /app
RUN pip install streamlit psycopg2-binary
CMD ["streamlit", "run", "streamlit_app.py"]
```

```
docker-compose up
```

```
docker ps
```

```
docker exec -it <container id> bash
```